# 【AIコース】大課題 毒キノコの判定 SVM

### ●【課題】SVMとは何か

答え:SVMは分類、回帰どちらの機械学習にも使用出来る機械学習の手法の一つである。2つのクラスを分割するための超平面を探す。  

__□SVMの仕組み__  

__-分類（線形）２次元の場合  __  
学習データがプロットされた領域を直線を引き２つに分ける。そして未知のデータ(testdata)に対して、そのデータが直線で分けたどちらの領域に存在するデータなのかを解釈して分類するというもの。  

__-分類（非線形）__  
学習データが線形では分類が困難な場合、カーネルトリックを用いて学習データを高次元へと移して分類する     例.2次元から3次元   
※カーネルとはふたつのベクトル x と y の内積を(たいていはとても高次元の)特徴空間で計算する方法。    

カーネルトリックの手法として下記の4つのカーネルがある。  
__・RBFカーネル__   
__・多項式カーネル__  
__・シグモイドカーネル__    
__・線形カーネル__ 線形分類で使用。  

 __□SVMの利点と欠点__  
 
 __-利点__  
 ・カーネルによって高次元へとデータを移す事で非線形問題を扱える。高い表現力を持つモデルが作れる。  
 ・データの特徴の次元（特徴量数）が大きくなっても精度が良い。  
 ・最適化するハイパーパラメータが少ない。  
 ・パラメータの産出が容易である。  
 
 
 __-欠点__  
 ・学習データが増えると計算量が膨大になる。「次元の呪い」の影響が顕著。  
 ・基本的には２クラスの分類にしか使えない。

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import display

### ●データの取得

In [2]:
df=pd.read_csv('Mushrooms.csv')
df.head()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


In [3]:
df.columns

Index(['class', 'cap-shape', 'cap-surface', 'cap-color', 'bruises', 'odor',
       'gill-attachment', 'gill-spacing', 'gill-size', 'gill-color',
       'stalk-shape', 'stalk-root', 'stalk-surface-above-ring',
       'stalk-surface-below-ring', 'stalk-color-above-ring',
       'stalk-color-below-ring', 'veil-type', 'veil-color', 'ring-number',
       'ring-type', 'spore-print-color', 'population', 'habitat'],
      dtype='object')

### ●前処理  
### 【課題】データの変換

In [4]:
# 要素の数を調べる
df['class'].value_counts()

e    4208
p    3916
Name: class, dtype: int64

In [5]:
df.columns

Index(['class', 'cap-shape', 'cap-surface', 'cap-color', 'bruises', 'odor',
       'gill-attachment', 'gill-spacing', 'gill-size', 'gill-color',
       'stalk-shape', 'stalk-root', 'stalk-surface-above-ring',
       'stalk-surface-below-ring', 'stalk-color-above-ring',
       'stalk-color-below-ring', 'veil-type', 'veil-color', 'ring-number',
       'ring-type', 'spore-print-color', 'population', 'habitat'],
      dtype='object')

# 文字列を数値に変換 LabelEncoderを使用。



In [21]:
from sklearn import preprocessing as sp

def features_Encode(features):
    for i in features:
        le = sp.LabelEncoder()
        le.fit(df[i].unique())
        df[i]=le.transform(df[i])
        return df[i]

features = ['class', 'cap-shape', 'cap-surface', 'cap-color', 'bruises', 'odor',
       'gill-attachment', 'gill-spacing', 'gill-size', 'gill-color',
       'stalk-shape', 'stalk-root', 'stalk-surface-above-ring',
       'stalk-surface-below-ring', 'stalk-color-above-ring',
       'stalk-color-below-ring', 'veil-type', 'veil-color', 'ring-number',
       'ring-type', 'spore-print-color', 'population', 'habitat']

features_Encode(features )
df.head()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,1,5,2,4,1,6,1,0,1,4,...,2,7,7,0,2,1,4,2,3,5
1,0,5,2,9,1,0,1,0,0,4,...,2,7,7,0,2,1,4,3,2,1
2,0,0,2,8,1,3,1,0,0,5,...,2,7,7,0,2,1,4,3,2,3
3,1,5,3,8,1,6,1,0,1,5,...,2,7,7,0,2,1,4,2,3,5
4,0,5,2,3,0,5,1,1,0,4,...,2,7,7,0,2,1,0,3,0,1


from sklearn import preprocessing as sp
le = sp.LabelEncoder()

le.fit(df[df.columns[0]].unique())
df['class']=le.transform(df[df.columns[0]])
le.fit(df[df.columns[1]].unique())
df['cap-shape']=le.transform(df[df.columns[1]])
le.fit(df[df.columns[2]].unique())
df['cap-surface']=le.transform(df[df.columns[2]])
le.fit(df[df.columns[3]].unique())
df['cap-color']=le.transform(df[df.columns[3]])
le.fit(df[df.columns[4]].unique())
df['bruises']=le.transform(df[df.columns[4]])
le.fit(df[df.columns[5]].unique())
df['odor']=le.transform(df[df.columns[5]])
le.fit(df[df.columns[6]].unique())
df['gill-attachment']=le.transform(df[df.columns[6]])
le.fit(df[df.columns[7]].unique())
df['gill-spacing']=le.transform(df[df.columns[7]])
le.fit(df[df.columns[8]].unique())
df['gill-size']=le.transform(df[df.columns[8]])
le.fit(df[df.columns[9]].unique())
df['gill-color']=le.transform(df[df.columns[9]])
le.fit(df[df.columns[10]].unique())
df['stalk-shape']=le.transform(df[df.columns[10]])
le.fit(df[df.columns[11]].unique())
df['stalk-root']=le.transform(df[df.columns[11]])
le.fit(df[df.columns[12]].unique())
df['stalk-surface-above-ring']=le.transform(df[df.columns[12]])
le.fit(df[df.columns[13]].unique())
df['stalk-surface-below-ring']=le.transform(df[df.columns[13]])
le.fit(df[df.columns[14]].unique())
df['stalk-color-above-ring']=le.transform(df[df.columns[14]])
le.fit(df[df.columns[15]].unique())
df['stalk-color-below-ring']=le.transform(df[df.columns[15]])
le.fit(df[df.columns[16]].unique())
df['veil-type']=le.transform(df[df.columns[16]])
le.fit(df[df.columns[17]].unique())
df['veil-color']=le.transform(df[df.columns[17]])
le.fit(df[df.columns[18]].unique())
df['ring-number']=le.transform(df[df.columns[18]])
le.fit(df[df.columns[19]].unique())
df['ring-type']=le.transform(df[df.columns[19]])
le.fit(df[df.columns[20]].unique())
df['spore-print-color']=le.transform(df[df.columns[20]])
le.fit(df[df.columns[21]].unique())
df['population']=le.transform(df[df.columns[21]])
le.fit(df[df.columns[22]].unique())
df['habitat']=le.transform(df[df.columns[22]])

## ●【課題】データセットの分割

In [22]:
# classの削除
df_X = df.drop('class',axis=1)
df_X.head()

,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,5,2,4,1,6,1,0,1,4,0,...,2,7,7,0,2,1,4,2,3,5
1,5,2,9,1,0,1,0,0,4,0,...,2,7,7,0,2,1,4,3,2,1
2,0,2,8,1,3,1,0,0,5,0,...,2,7,7,0,2,1,4,3,2,3
3,5,3,8,1,6,1,0,1,5,0,...,2,7,7,0,2,1,4,2,3,5
4,5,2,3,0,5,1,1,0,4,1,...,2,7,7,0,2,1,0,3,0,1


In [23]:
df_y = df.loc[:,['class']]
df_y.head()

,class
0,1
1,0
2,0
3,1
4,0


In [24]:
X = df_X
y = df_y

In [25]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=0)

## ●【課題】標準化

In [26]:
# 標準化
from sklearn.preprocessing import StandardScaler
stdsc = StandardScaler()

#訓練データのみの平均、標準偏差を用いて標準化する。
X_train_std = stdsc.fit_transform(X_train)
#テストデータも標準化する。
X_test_std = stdsc.transform(X_test)

print(X_train_std.mean())
print(X_train_std.std())
# test用はtrainを基準に標準化してるので、多少0,1からずれる
print(X_test_std.mean())
print(X_test_std.std())

-4.09991577027e-19
0.977008420918
-0.00569082312094
0.991139328626


In [27]:
X_train_max=pd.DataFrame(X_train_std,columns=['cap-shape', 'cap-surface', 'cap-color', 'bruises', 'odor',
       'gill-attachment', 'gill-spacing', 'gill-size', 'gill-color',
       'stalk-shape', 'stalk-root', 'stalk-surface-above-ring',
       'stalk-surface-below-ring', 'stalk-color-above-ring',
       'stalk-color-below-ring', 'veil-type', 'veil-color', 'ring-number',
       'ring-type', 'spore-print-color', 'population', 'habitat'])
X_train_max.head()

,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,-0.213438,0.143646,-0.580420,-0.841703,0.402885,0.156309,2.263924,-0.673491,0.615424,-1.148658,...,0.584069,0.620051,0.629656,0.0,0.133889,3.400235,0.955810,1.423856,-0.504027,-0.296749
1,1.035105,-1.478513,1.398050,-0.841703,0.402885,0.156309,2.263924,-0.673491,-0.794091,-1.148658,...,0.584069,0.620051,0.629656,0.0,0.133889,3.400235,0.955810,1.423856,-1.296210,-0.296749
2,1.035105,0.143646,1.398050,1.188067,-0.553765,0.156309,-0.441711,-0.673491,0.051618,-1.148658,...,0.584069,0.620051,0.629656,0.0,0.133889,-0.256579,0.955810,-0.673714,-0.504027,0.863293
3,-0.837709,0.143646,1.398050,-0.841703,0.402885,0.156309,2.263924,-0.673491,-0.512188,0.870581,...,-2.364175,0.620051,0.629656,0.0,0.133889,-0.256579,-1.259923,-0.673714,-0.504027,-0.296749
4,-0.213438,0.954725,-0.976114,-0.841703,1.837861,0.156309,-0.441711,1.484801,-1.357896,0.870581,...,-0.890053,0.620051,0.104485,0.0,0.133889,-0.256579,-1.259923,1.423856,0.288155,0.283272


__※なぜSVMを行う際に標準化が必要なのか?__

A.標準化が必要な理由は、それぞれの特徴量のスケールが異なる（例えば、身長や体重、家の価格と部屋数ではその値を表す単位と範囲が異なる。）場合にモデルをうまく学習させられない可能性がある。なので特徴量間で値のスケールを揃える必要がある。この作業を標準化という。

## ●ハイパーパラメータの調整

### __【課題】カーネルとは何か__

A.カーネルとは、SVMに於いて非線形の、線形では分けられない複雑な問題に於いて高次元へと移し分類する手法である。カーネルトリックとも言う。  
カーネルは主にRBFカーネルが精度が高く一般的に使われている。

### __【課題】コストペナルティCとは何か__

A.コストペナルティCとは、どのぐらい誤分類を許容するかを決めるパラメータである。Cが小さい程誤分類を許容するように、大きい程誤分類を許容しないように超平面を決定する。

### __【課題】ハイパーパラメータを調整する__

In [28]:
# グリッドサーチの際にはy＿trainをnp.arrayにする必要がある。
X_train_std.shape
new_y_train = np.array(y_train.values.flatten())
new_y_train.shape

(6499,)

In [29]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

tuned_parameters = [{'C': [0.1,1,10], 'kernel': ['rbf']},{'C': [0.1,1,10], 'kernel': ['poly']},{'C': [0.1,1,10], 'kernel': ['sigmoid']}]

clf= GridSearchCV(SVC(),tuned_parameters, cv=5)

clf.fit(X_train_std,new_y_train)

print(clf.grid_scores_)
print(clf.best_params_)

[mean: 0.98969, std: 0.00302, params: {'C': 0.1, 'kernel': 'rbf'}, mean: 1.00000, std: 0.00000, params: {'C': 1, 'kernel': 'rbf'}, mean: 1.00000, std: 0.00000, params: {'C': 10, 'kernel': 'rbf'}, mean: 0.99446, std: 0.00113, params: {'C': 0.1, 'kernel': 'poly'}, mean: 1.00000, std: 0.00000, params: {'C': 1, 'kernel': 'poly'}, mean: 1.00000, std: 0.00000, params: {'C': 10, 'kernel': 'poly'}, mean: 0.90660, std: 0.00328, params: {'C': 0.1, 'kernel': 'sigmoid'}, mean: 0.82828, std: 0.00919, params: {'C': 1, 'kernel': 'sigmoid'}, mean: 0.80966, std: 0.01134, params: {'C': 10, 'kernel': 'sigmoid'}]
{'C': 1, 'kernel': 'rbf'}


/Users/takahashikoji/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/site-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


### __【課題】学習およびモデル評価__

### ●RBFカーネルでの学習

In [30]:
rbf_svm = SVC(kernel='rbf' , gamma='auto' , C=1.0)
rbf_svm.fit(X_train_std,new_y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

### ●モデル評価

### 決定係数 [score]

In [31]:
rbf_svm.score(X_test_std,y_test)

1.0

### 正解率 [Accuracy]

In [32]:
from sklearn.metrics import accuracy_score

y_pred = rbf_svm.predict(X_test_std)
print(y_pred)
accuracy_score(y_test,y_pred)

[1 0 0 ..., 1 0 0]


1.0

### [Confusion Matrix]

__どの正解ラベルのデータをどのラベルに何個分類したかという表。__   
__対角線上が正解した数(1を1、0を0と分類)となっていて、その他が間違いの数(1を0、0を1と分類)になっている。今回は全て正解。__

In [33]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test,y_pred)

array([[852,   0],
       [  0, 773]])

### [Classification Report]

__この関数はPrecision、RecallとF値とsupport(正解ラベルのデータの数)を教えてくれる。__  
__Precision、Recall、F値は評価に非常によく使われています。__

In [34]:
from sklearn.metrics import classification_report

print(classification_report(y_test,y_pred))

             precision    recall  f1-score   support

          0       1.00      1.00      1.00       852
          1       1.00      1.00      1.00       773

avg / total       1.00      1.00      1.00      1625

